# 进展记录
该脚本主要记录实验进展，包括但不限于实验设置、实验结果、分析、疑惑记录等内容

# 11月

## 11.26
### 记录
`logs/rl_games/leaphand_object_rot/2025-11-26_10-28-35` 关节空间
<br> `logs/rl_games/leaphand_se3_rot/2025-11-26_10-03-32` se3末端刚体旋量空间
<br> `logs/rl_games/leaphand_se3_rot/2025-11-26_21-48-16` se3缩放修正版
#### 分析与疑惑
- 试验了se3动作空间的训练效果，初步结果不太理想。主要问题在于奖励收敛很慢，且最终效果不如关节空间动作。怀疑原因有：1是se3动作空间的高维度和冗余性导致训练难度加大；2是当前的奖励设计和观察空间并未针对se3动作进行优化，导致学习效率低下。
- 先做了简单的对比训练，除了动作项外其他都一致。se3动作能做，但奖励收敛很慢。同时发现了一些奇怪的现象：1是se3手指和物体不怎么接触，很多是靠指腹带动物体转；2是实际关节动作抖动性不算太大，但经常一阵一阵的反复，不知道是se(3)旋量和J_b的伪逆导致的这种不稳定行为不。
- 传统的EMA，是在神经网络外部显示裁减增量幅度来达到动作平滑化，并非网络内生平滑。但我目前如果在深度学习这块去做，不知道来不来得及，但实现神经网络的物理内生一致性，达到对于大多数任务所需要的平滑一致动作输出，确实是个潜在的idea
- 虽然初次训练效果一般，但我仍坚信se(3)的应用潜力。也许每个动作表征空间都有自身适用的机制，它们的偏好分布具有偏差，强制相同条件下去比对也未必公平。或许当下阶段，不是在当前关节空间最合适的配置里去调se3旋量动作空间的效果，而是从se3本身出发，去调整适配于se3自身的观察、奖励等，以达到效果更好。真正的比较应该是SOTA比SOTA，关节空间SOTA比se3旋量空间SOTA
- 非常奇怪，修改了se3actions.py里的缩放，但训练效果反而更差了，不如之前的版本。怀疑是哪里有bug，待排查
> 更正：缩放修正后后期奖励比未修正的要好，最后阶段的奖励是未修正时的4倍左右，可能只是前面慢一点；
### 计划
开始做针对se3的适配工作，分为5大块
1. 设置适合se(3)的观察空间。
<br>感觉关节量可能不是很合适，因为这相当于仍要学习一个正向运动学的过程构建，网络会隐式地包含手的结构信息。但本体感知的信息本身就很少，关节量适合用作基准参照。
难道说，这时的本体感知变成了目标旋量动作？
理想中更合适的相对信息，来自视触觉模态信息。一种是点云，手和物体点云所构成的相对信息。一种是触觉，如视触觉。
重点可探索点云方面的视觉信息，最好是即时可用的。
IDEA：不是说结构信息不必学，是想让策略网络更关注任务本身的层面。结构这块的信息，或许可以参照GET-Zero训练一个上游特征的结构提取基座大模型G: q -->q* ，但怎么去理解手的结构本身就是一个难题
1. 奖励函数设计的调整 (Crucial for se3)
<br>关于动作稳定和平滑，可以改为惩罚SE(3)能量 ${\mathcal{V}_b^{T}}M\mathcal{V}_b$ ，以及 。
关于操作多样性，可用可操作度指标 (Manipulability Index) $det(J{{J}^{T}})$ 或 雅可比条件数 $\frac{{{\sigma }_{max}}(J)}{{{\sigma }_{min}}(J)}$ 。
1. 动作的微调
<br>先想的是额外再添加一个动作缩放因子  
可选：自适应 $\lambda$，具体实现方式待定；PD控制中引入位置误差补偿（Closed-Loop Inverse Kinematics, CLIK），有无必要确认：
1. 网络的设置
<br>现有的动作维度为24，以及考虑到如果用关节作为本体感知信息，那终究还是要隐式的学习手的动力学结构，而手指为4个节点的树，那是否意味着用4层MLP会更好？还有动作空间匹配的问题
1. 学习算法
<br>不知道rl_games最新的pbt训练是否对训练效果的提升或者超参数的调整有所增益？SAC算法和TD3算法和PPO相比又如何

## 11.27
### 计划与行动
今天主要针对奖励这块修正。昨天的实验，根据奖励管理器的记录，我觉得pose_diff权重偏高，而且其设置的默认位形可能比较奇异，我感觉是不太好的。关于pose_diff，我可以重新人为设置和调整下更合适的初始位形，甚至有一个
IDEA：利用别人预训练好的抓取网络，生成一个抓取姿态反解得到初始位形（这样可以保证手指和物体有良好的接触、且可针对不同物体进行泛化）。后期训练过程中可以通过课程学习逐渐降低pose_diff的权重。

1.奖励的设计与调整
<br>主要想调整pose_diff和操作度指标这两块，思路和昨天设想的一致。
<br>